In [1]:
import os
import pandas as pd
from pathlib import Path
from harp_resources import process, utils
from analysis_functions import *

In [2]:
mouse_info = {'B2M1': {'sex': 'M', 'area': 'V2M'},
              'B2M4': {'sex': 'M', 'area': 'V2M'},
              'B2M5': {'sex': 'M', 'area': 'V2M'},
              'B2M6': {'sex': 'M', 'area': 'V2M'},
              'B3M1': {'sex': 'M', 'area': 'V2M'},
              'B3M2': {'sex': 'M', 'area': 'V2M'},
              'B3M3': {'sex': 'F', 'area': 'V1'},
              'B3M4': {'sex': 'M', 'area': 'V2M'},
              'B3M5': {'sex': 'M', 'area': 'V2M'},
              'B3M6': {'sex': 'F', 'area': 'V2M'},
              'B3M7': {'sex': 'F', 'area': 'V2M'},
              'B3M8': {'sex': 'F', 'area': 'V2M'},
             }

session_info = {'220824': 'day1',
                '230824': 'day2',
                '190824': 'day1',
                '200824': 'day2',
                '120824': 'day1',
                '130824': 'day2',
                '070824': 'day1',
                '080824': 'day2',
               }

## Defining paths for grab or G8

In [3]:
rootdir = '/Volumes/RanczLab/20240730_Mismatch_Experiment/GRAB_MMclosed-and-open_190824' #Enter root path
h5_paths = []
eventpaths = []
for dirpath, subdirs, files in os.walk(rootdir):
    for x in files:
        if '.h5' in x:
            eventpaths.append(dirpath)
            h5_paths.append(dirpath+'/'+x)

### Loading data streams

In [4]:
stream_dict_dict = load_h5_streams_to_dict(h5_paths)

reconstructing streams for mouse B2M6, from session folder: GRAB_MMclosed-and-open_190824
  --> B2M6 streams reconstructed and added to dictionary 

reconstructing streams for mouse B3M8, from session folder: GRAB_MMclosed-and-open_190824
  --> B3M8 streams reconstructed and added to dictionary 

reconstructing streams for mouse B3M7, from session folder: GRAB_MMclosed-and-open_190824
  --> B3M7 streams reconstructed and added to dictionary 

reconstructing streams for mouse B3M6, from session folder: GRAB_MMclosed-and-open_190824
  --> B3M6 streams reconstructed and added to dictionary 

reconstructing streams for mouse B3M4, from session folder: GRAB_MMclosed-and-open_190824
  --> B3M4 streams reconstructed and added to dictionary 



In [14]:
'SleapVideoData2' in stream_dict_dict['B2M6']

False

In [15]:
data_dict = {}
for mouse, streamdict in stream_dict_dict.items():
    #Getting fluorescence traces
    fluorescence = streamdict['Photometry']['470_dfF'] #Using '470_dfF' only

    #Getting mouse movement data and converting to cm / second
    movementX = process.running_unit_conversion(streamdict['H1']['OpticalTrackingRead0X(46)'])*100
    movementY = process.running_unit_conversion(streamdict['H1']['OpticalTrackingRead0Y(46)'])*100

    #Getting eye movements and pupil diameter
    if 'SleapVideoData2' in streamdict:
        eye_center_x = streamdict['SleapVideoData2']['Ellipse.Center.X']
        eye_center_y = streamdict['SleapVideoData2']['Ellipse.Center.Y']
        eye_diameter = streamdict['SleapVideoData2']['Ellipse.Diameter']
    else: 
        print('There was no eye movement data available for ', mouse)

    #Getting visual stimuli event times
    event = streamdict['ONIX']['Photodiode']
    
    time = movementX.index - movementX.index[0]
    
    dict = {'470_dfF': fluorescence, 'movementX': movementX, 'movementY': movementY, 'event': event,
        'Seconds': time}
    #dict = {'470_dfF': fluorescence, 'movementX': movementX, 'movementY': movementY, 'event': event,
     #   'TimeStamp': time, 'eye_x': eye_center_x, 'eye_y': eye_center_y, 'pupil_diameter': eye_diameter}

    df = pd.DataFrame(dict)

    #Reversing, so that a halt appearst when 'event'==True
    df['event'] = ~df['event']
    
    df.reset_index(inplace=False)
    
    data_dict[mouse] = df

There was no eye movement data available for  B2M6
There was no eye movement data available for  B3M8
There was no eye movement data available for  B3M7
There was no eye movement data available for  B3M6
There was no eye movement data available for  B3M4


In [16]:
data_dict['B3M7']

,470_dfF,movementX,movementY,event,Seconds
95487.9626,0.000000,0.000000,0.000000,True,0.0000
95487.9627,0.000342,0.000743,0.000203,True,0.0001
95487.9628,0.000683,0.001486,0.000405,True,0.0002
95487.9629,0.001025,0.002228,0.000608,True,0.0003
95487.9630,0.001366,0.002971,0.000811,True,0.0004
...,...,...,...,...,...
97423.4075,0.000052,0.000268,-0.000061,False,1935.4449
97423.4076,0.000039,0.000201,-0.000046,False,1935.4450
97423.4077,0.000026,0.000134,-0.000031,False,1935.4451
97423.4078,0.000013,0.000067,-0.000015,False,1935.4452


### Loading Experiment events and session info

In [17]:
event_dict = {}
for eventpath in eventpaths:
    ExpEvents = read_ExperimentEvents(Path(eventpath))
    ExpEvents.set_index('Seconds', inplace = True)
    ExpEvents.index = ExpEvents.index.round(4)
    name = eventpath.split('/')[-1][-4:]
    ExpEvents['experiment'] = eventpath.split('/')[-2].split('_')[1]
    for key, item in session_info.items():
        if key in eventpath.split('/')[-2]:
            ExpEvents['session']=item
    event_dict[name] = ExpEvents


### Adding events (and non-events) and session info to data

In [18]:
event_dict['B3M6'].loc[event_dict['B3M6'].Value == 'LinearPlaybackMismatch block started']

,Value,experiment,session
Seconds,,,
98993.4863,LinearPlaybackMismatch block started,MMclosed-and-open,day1


In [23]:
data_dict = add_experiment_events(data_dict, event_dict,mouse_info)

Pre-existing ExperimentEvents column was replaces with new for B2M6
Pre-existing ExperimentEvents column was replaces with new for B3M8
Pre-existing ExperimentEvents column was replaces with new for B3M7
Pre-existing ExperimentEvents column was replaces with new for B3M6
Pre-existing ExperimentEvents column was replaces with new for B3M4


In [26]:
data_dict['B3M6'].ExperimentEvents.unique()

array([nan, 'Sync signal started', 'LinearMismatch block started',
       'Homing platform', 'Wait for run threshold...',
       'Halt delay: 0.597427398570547s', 'Apply halt: 1s',
       'Halt delay: 0.555732677344108s', 'Halt delay: 0.435164502419142s',
       'Halt delay: 0.567310379709727s', 'Halt delay: 0.288090241834563s',
       'Halt delay: 0.512158285692408s', 'Halt delay: 0.537869750167183s',
       'Halt delay: 0.324849802313768s', 'Halt delay: 0.127083636506965s',
       'Halt delay: 0.273953280166701s', 'Halt delay: 0.285591537824642s',
       'Halt delay: 0.380470190467532s', 'Halt delay: 0.329870850327365s',
       'Halt delay: 0.174229225318054s', 'Halt delay: 0.29355418961195s',
       'Halt delay: 0.377370904235808s', 'Halt delay: 0.590421244171644s',
       'Halt delay: 0.262138870759932s', 'Halt delay: 0.11783661335606s',
       'Halt delay: 0.180272276690357s', 'Halt delay: 0.350562733155937s',
       'Halt delay: 0.266652444827674s', 'Halt delay: 0.57921669458934s

In [32]:
data_dict = add_no_halt_column(data_dict, event_dict)

No_halt events added to B2M6
Correct number of no-halt events for B2M6
No_halt events added to B3M8
Correct number of no-halt events for B3M8
No_halt events added to B3M7
Correct number of no-halt events for B3M7
No_halt events added to B3M6
Correct number of no-halt events for B3M6
No_halt events added to B3M4
Correct number of no-halt events for B3M4


In [43]:
for name, df in data_dict.items():
    print('updating data for ', name)
    blocks_added_df = add_block_columns(df, event_dict[name])
    blocks_added_df.replace({})
    data_dict[name] = blocks_added_df

check_block_overlap(data_dict)

updating data for  B2M6
LinearMismatch_block
LinearPlaybackMismatch_block
updating data for  B3M8
LinearMismatch_block
LinearPlaybackMismatch_block
updating data for  B3M7
LinearMismatch_block
LinearPlaybackMismatch_block
updating data for  B3M6
LinearMismatch_block
LinearPlaybackMismatch_block
updating data for  B3M4
LinearMismatch_block
LinearPlaybackMismatch_block


In [57]:
def downsample_data(df, time_col='Seconds', interval=0.001):
    # Convert the Seconds column to a TimedeltaIndex
    df = df.set_index(pd.to_timedelta(df[time_col], unit='s'))

    # Define aggregation functions for all possible columns
    aggregation_functions = {
        '470_dfF': 'mean',
        'movementX': 'mean',
        'movementY': 'mean',
        'event': 'any',
        'ExperimentEvents': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
        'Experiment': 'first',
        'Session': 'first',
        'mouseID': 'first',
        'sex': 'first',
        'area': 'first',
        'No_halt': 'any',
        'LinearMismatch_block': 'any',
        'LinearPlaybackMismatch_block': 'any',
        'LinearRegular_block': 'any'
    }

    # Filter aggregation_functions to only include columns present in df
    aggregation_functions = {key: func for key, func in aggregation_functions.items() if key in df.columns}

    # Resample with the specified interval and apply the filtered aggregations
    downsampled_df = df.resample(f'{interval}s').agg(aggregation_functions)

    # Reset the index to make the Seconds column normal again
    downsampled_df = downsampled_df.reset_index()
    downsampled_df[time_col] = downsampled_df[time_col].dt.total_seconds()  # Convert Timedelta back to seconds

    # Forward fill for categorical columns if needed, only if they exist in downsampled_df
    categorical_cols = ['Experiment', 'Session', 'mouseID', 'sex', 'area']
    for col in categorical_cols:
        if col in downsampled_df.columns:
            downsampled_df[col] = downsampled_df[col].ffill()

    # Remove consecutive duplicate values in the 'ExperimentEvents' column, if it exists
    if 'ExperimentEvents' in downsampled_df.columns:
        downsampled_df['ExperimentEvents'] = downsampled_df['ExperimentEvents'].where(
            downsampled_df['ExperimentEvents'] != downsampled_df['ExperimentEvents'].shift()
        )

    return downsampled_df



In [58]:
def test_event_numbers(downsampled_data, original_data, mouse):
    nohalt_down = len(downsampled_data.loc[downsampled_data['No_halt']==True])
    nohalt_original = len(original_data.loc[original_data['No_halt']==True])
    if nohalt_down != nohalt_original:
        print(f'mouse{mouse}')
        print(f'There are actually {nohalt_original} no-halts, but the downsampled data only contains {nohalt_down}')
    
    

In [ ]:
'''downsampled_dict = {}
for mouse, mouse_df in data_dict.items():
    downsampled_df = downsample_data(mouse_df, time_col='Seconds', interval=0.001)
    downsampled_dict[mouse]=downsampled_df
    test_event_numbers(downsampled_df, mouse_df, mouse)
    '''
mouse = 'B3M6'
downsampled_df = downsample_data(data_dict[mouse], time_col='Seconds', interval=0.001)
test_event_numbers(downsampled_df, data_dict[mouse], mouse)

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(downsampled_df['Seconds'].loc[downsampled_df['event']==True], downsampled_df['470_dfF'].loc[downsampled_df['event']==True])

In [ ]:

downsampled_df.ExperimentEvents.unique()#.loc[downsampled_df.No_halt==True]


In [ ]:
downsampled_df#.loc[downsampled_df.No_halt==True]

downsampled_df.ExperimentEvents.unique()#.loc[downsampled_df.No_halt==True]
downsampled_df.loc[downsampled_df.ExperimentEvents=='Apply halt: 1s']


In [ ]:
downsampled_df.loc[downsampled_df.Seconds >87.4]

In [ ]:
downsampled_df.loc[downsampled_df.event ==True]

In [ ]:
#Data = process.pooling_data(data_dict)


In [ ]:
Data = Data.reset_index()
Data = Data.drop(columns=['level_0'])  # Assuming the column name is 'level_0' after reset_index()
Data = Data.set_index('level_1')  # 'level_1' will be the numeric index part
Data.index.name = 'Time'

In [ ]:
import matplotlib.pyplot as plt

plt.plot()

In [ ]:
#20240730_Mismatch_Experiment/GRAB_MMclosed-and-open_190824'
#Data.to_csv('GRAB_MMclosed_open_session1.csv', index=False)

In [ ]:
B3M7 = Data.loc[Data.mouseID=='B3M7']

In [ ]:
B3M4 = Data.loc[Data.mouseID=='B3M4']

In [ ]:
B3M4.loc[B3M4.ExperimentEvents == 'Block timer elapsed']

In [ ]:

B3M4.loc[B3M4.LinearPlaybackMismatch_block == True]